In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [ ]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']


In [ ]:
import os
import sys
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import googletrans
from pathlib import Path
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.pipeline import Sentencizer


In [ ]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [ ]:
df_jobs = df_jobs[:100]

In [ ]:
# df_jobs = df_jobs.sample(n=100, random_state=1)

In [ ]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [ ]:
df_jobs.info()


In [ ]:
df_jobs.columns


In [ ]:
str_fix_incl = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'
str_fix_eg = 'Partner with Procurement in order to manage suppliers for the projects & programs in scope, e.g.'



In [ ]:
len(df_jobs.loc[(df_jobs['Job Description'].str.contains(str_fix_incl)) | (df_jobs['Job Description'].str.contains(str_fix_eg))])


In [ ]:
# job_descriptions = list(set(df_jobs_incl['Job Description'].to_list()))


In [ ]:
job_descriptions = list(
    set(
        df_jobs['Job Description'].loc[
            (df_jobs['Job Description'].str.contains(str_fix_incl)) |
            (df_jobs['Job Description'].str.contains(str_fix_eg))
        ].to_list()
    )
)


In [ ]:
len(job_descriptions)


In [ ]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize
nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)


In [ ]:
nltk_sentencizer = []

for job_description in job_descriptions:
    doc = sent_tokenize(job_description)
    nltk_sentencizer.extend(doc)

In [ ]:
len(nltk_sentencizer)

In [ ]:
for idx, sent in enumerate(nltk_sentencizer):
    if str_fix_eg in sent:
        print(idx)

In [ ]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(nltk_sentencizer):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

In [ ]:
# sentences split on e.g. and incl.
nltk_sentencizer[8:13]

In [ ]:
for idx, sentence in enumerate(nltk_sentencizer[8:13]):
    print(f'Sentence {idx+1}: {sentence}\n')

In [ ]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars = [':', '|'], repeated_punct_chars = ['\n', ',']):
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [ ]:
custom_punct_chars = make_custom_punct_chars()

In [ ]:
custom_punct_chars

In [ ]:
# Set up Spacy
import spacy
from spacy.symbols import ORTH, LEMMA

nlp = spacy.load('en_core_web_sm')


In [ ]:
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)


In [ ]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    with open(f'{data_dir}punctuations.txt', 'wb') as f:
        pickle.dump(sentencizer.punct_chars, f)

In [ ]:
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_char = pickle.load(f)

In [ ]:
# special_cases = {":)": [{"ORTH": ":)"}]}
# prefix_re = re.compile(r'''^[\\[\\("']''')
# suffix_re = re.compile(r'''[\\]\\)"']$''')
# infix_re = re.compile(r'''[-~]''')
# simple_url_re = re.compile(r'''^https?://''')

# def custom_tokenizer(nlp):
#     return Tokenizer(nlp.vocab, rules=special_cases,
#                                 prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 url_match=simple_url_re.match)

In [ ]:
# nlp = spacy.load('en_core_web_sm')
# nlp.tokenizer.add_special_case('incl.', [{ORTH: 'incl', LEMMA: 'including'}])


In [ ]:
# from spacy.symbols import ORTH, LEMMA

special_cases_dict = {
    'incl.': [{65: 'incl', 67: 'including'}],
    'incl. ': [{65: 'incl', 67: 'including'}],
    '(incl.': [{65: 'incl', 67: 'including'}],
    'etc.': [{65: 'etc', 67: 'et cetera'}],
    'etc. ': [{65: 'etc', 67: 'et cetera'}],
    'e.g.': [{65: 'e.g', 67: 'for example'}],
    'e.g. ': [{65: 'e.g', 67: 'for example'}],
}

nlp.tokenizer.rules.update(special_cases_dict)


In [ ]:
nlp.tokenizer.rules['e.g.']

In [ ]:
# Spacy sentencizer
spacy_sentencizer = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_description in job_descriptions:
#         doc = nlp(job_description)
        spacy_sentencizer.extend([sent.text for sent in nlp(job_description).sents])


In [ ]:
job_descriptions[0][1500:2000]

In [ ]:
len(spacy_sentencizer)

In [ ]:
for idx, sent in enumerate(spacy_sentencizer):
    if str_fix_eg in sent:
        print(idx)

In [ ]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(spacy_sentencizer):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

In [ ]:
spacy_sentencizer[9:13]

In [ ]:
for idx, sentence in enumerate(spacy_sentencizer[9:13]):
    print(f'Sentence {idx+1}: {sentence}\n')

In [ ]:
# Spacy tokenizer
spacy_tokenizer = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_sentence in spacy_sentencizer:
#         doc = nlp.tokenizer(job_sentence)
        spacy_tokenizer.extend(
            [
                token.text for token in nlp.tokenizer(job_sentence) 
                if token.text not in custom_punct_chars
                and not token.is_stop 
                
            ]
        )

In [ ]:
len(spacy_tokenizer)

In [ ]:
spacy_tokenizer[:10]

In [ ]:
# from spacy.lang.en import English
# nlp = English()
# nlp.add_pipe('sentencizer')
doc = nlp('Operating income incl. JV was SEK 2.1 b. with an operating margin of 4.0%')

In [ ]:
[s for s in doc.sents]

In [ ]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_trf')

matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

trigram_patterns = [[{"POS": i} for i in j] for j in rules]


In [ ]:
patterns

In [ ]:
patterns = {
    'noun_verb': [{'POS': 'NOUN'}, {'POS': 'VERB'}],
    'verb_noun': [{'POS': 'VERB'}, {'POS': 'NOUN'}],
    'adj_noun': [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
    'adj_propn': [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
}

In [ ]:
for pattern_name, pattern in patterns.items():
    matcher.add(pattern_name, [pattern])


In [ ]:
for job_description in job_descriptions:
    doc = nlp(job_description)
    matches = matcher(doc)

In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(span.text))
    print(match_id, string_id, start, end)
    print()


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
# for sentence in spacy_sentencizer:
#     print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)